##Imports

In [20]:
import re
import pandas as pd
import json
import nltk
import numpy as np

#Imports similaridade
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/lucas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/lucas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Extração de Dados

In [10]:
def df_tweets_candidatos(json_filename):
    df = pd.read_json(json_filename).drop(columns = ['replies'])
    return df


def df_tweets_respostas(json_filename):
    df = pd.read_json(json_filename).dropna().reset_index()[['replies']]
    ds = []
    for replies in df['replies'].to_list():
        for reply in replies:
            ds.append(reply)

    reply_df = pd.DataFrame (ds).drop(columns = ['attachments'])
    return reply_df

def df_tweets_cadidatos_respostas(json_filename):
    df_tweets_cand = df_tweets_candidatos(json_filename)
    df_tweets_reps = df_tweets_respostas(json_filename)

    return (df_tweets_cand, df_tweets_reps)

In [13]:
df2 = df_tweets_cadidatos_respostas('datasets/ciro-result.json')
df2[0].head()

,text,created_at,author_id,id,conversation_id
0,Todo mundo viu esse papo de altíssimo nível co...,2022-08-19 01:31:22+00:00,33374761,1560439246913257472,1560439246913257472
1,É isso que a gente necessita: desarmar. Precis...,2022-08-18 22:41:15+00:00,33374761,1560396436294062080,1560396434725376000
2,Pare pra pensar e me responda: se Bolsonaro ou...,2022-08-18 22:41:15+00:00,33374761,1560396434725376000,1560396434725376000
3,Ninguém tem direito de explorar o jeito do pov...,2022-08-18 22:38:57+00:00,33374761,1560395854887936000,1560395854887936000
4,Dizer que eu não votei em 2018 é picaretagem a...,2022-08-18 22:01:32+00:00,33374761,1560386438398898176,1560386438398898176


# Pré-Processamento

In [14]:
def normaliza(df, coluna_alvo='text'):
  coluna_normal = coluna_alvo+"_normal"
  df[coluna_normal] = df[coluna_alvo].str.lower()
  df[coluna_normal].replace(to_replace = '[^a-zA-ZÀ-ÖØ-öø-ÿ\s]', value='', regex = True, inplace = True)
  return df
  
def tokeniza(df, coluna_alvo='text_normal'):
  stop_words = nltk.corpus.stopwords.words('portuguese')
  todos_tokens = []
  todos_textos_tokens = []
  ind = 0

  for texto_normal in df[coluna_alvo].to_list():
    tokens = nltk.word_tokenize(texto_normal, language="portuguese")
    tokens_filtrados = [token for token in tokens if token not in stop_words]
    todos_tokens.append(tokens_filtrados)
    texto_tokens = ' '.join(tokens_filtrados)
    todos_textos_tokens.append(texto_tokens)
    ind += 1

  df_tokens = pd.DataFrame({'tokens': todos_tokens, 'texto_tokens': todos_textos_tokens})
  df_concat = pd.concat([df, df_tokens], axis="columns")

  return df_concat


def normaliza_tokeniza(df, coluna_alvo = 'text'):
  df_tokens = tokeniza(normaliza(df, coluna_alvo), coluna_alvo)
  return df_tokens

  #word_regex = r"[-'a-zA-ZÀ-ÖØ-öø-ÿ]+" #para capturarmos palavras dentro do tweet
  #df['text'] = re.sub(r'^https?:\/\/.*[\r\n]*', '', df['text'], flags=re.MULTILINE)  #para remover os links do campo de texto do tweet

,text,created_at,author_id,id,conversation_id,text_normal,tokens,texto_tokens
0,Todo mundo viu esse papo de altíssimo nível co...,2022-08-19 01:31:22+00:00,33374761,1560439246913257472,1560439246913257472,todo mundo viu esse papo de altíssimo nível co...,"[Todo, mundo, viu, papo, altíssimo, nível, que...",Todo mundo viu papo altíssimo nível querido @ ...
1,É isso que a gente necessita: desarmar. Precis...,2022-08-18 22:41:15+00:00,33374761,1560396436294062080,1560396434725376000,é isso que a gente necessita desarmar precisam...,"[É, gente, necessita, :, desarmar, ., Precisam...",É gente necessita : desarmar . Precisamos reco...
2,Pare pra pensar e me responda: se Bolsonaro ou...,2022-08-18 22:41:15+00:00,33374761,1560396434725376000,1560396434725376000,pare pra pensar e me responda se bolsonaro ou ...,"[Pare, pra, pensar, responda, :, Bolsonaro, Lu...",Pare pra pensar responda : Bolsonaro Lula venc...
3,Ninguém tem direito de explorar o jeito do pov...,2022-08-18 22:38:57+00:00,33374761,1560395854887936000,1560395854887936000,ninguém tem direito de explorar o jeito do pov...,"[Ninguém, direito, explorar, jeito, povo, ador...",Ninguém direito explorar jeito povo adorar Deu...
4,Dizer que eu não votei em 2018 é picaretagem a...,2022-08-18 22:01:32+00:00,33374761,1560386438398898176,1560386438398898176,dizer que eu não votei em é picaretagem absol...,"[Dizer, votei, 2018, picaretagem, absoluta, .,...",Dizer votei 2018 picaretagem absoluta . O fiz ...


# Modelagem de Tópicos

*texto em itálico*# Análise de Sentimentos

# Similaridade de Textos

In [16]:
def monta_docs(df, coluna_alvo, grupos, candidatos):
    docs = []
    for grp in grupos:
        for cand in candidatos:
            df_filtrado = df[df['topico'] == grp and df['candidato'] == cand]
            texto = ' '.join(df_filtrado[coluna_alvo].to_list())
            docs.append(texto)
 
    return docs

def vetoriza_docs(lista_docs):
    corpus_vetorizado = np.vectorize(lista_docs)
    return corpus_vetorizado

def gera_matriz_tfidf(corpus_vetorizado):
    tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
    matriz_tfidf = tf.fit_transform(corpus_vetorizado)
    return matriz_tfidf

def similaridade_pares(matriz_tfidf):
    dict_docs_sim = cosine_similarity(matriz_tfidf)
    df_sim = pd.DataFrame(dict_docs_sim)
    return df_sim

def calcula_similaridade(df, coluna_alvo, grupos, candidatos):
    lista_docs = monta_docs(df, coluna_alvo, grupos, candidatos)
    corpus_vetorizado = vetoriza_docs(lista_docs)
    matriz_tfidf = gera_matriz_tfidf(corpus_vetorizado)
    df_sim = similaridade_pares(matriz_tfidf)

    return df_sim


NameError: name 'norm_corpus' is not defined